In [1]:
import requests
import pandas as pd
import json
import requests
import re
from datetime import datetime
from bs4 import BeautifulSoup

In [6]:
import requests
from bs4 import BeautifulSoup
from _DESAdapter import DESAdapter, environment_requires_DES_adapter
import mechanicalsoup

def login(email, password):
    """
    Logs in to kenpom.com using user credentials and returns an authenticated session.
    
    Args:
        email (str): User e-mail for login to kenpom.com.
        password (str): User password for login to kenpom.com.
        
    Returns:
        session (requests.Session): Authenticated session with full access to kenpom.com.
    """

    # Fix for Cloudflare SSL profiling
    session = requests.Session()
    if environment_requires_DES_adapter():
        session.mount('https://kenpom.com/', DESAdapter())

    # Create a StatefulBrowser to manage login
    browser = mechanicalsoup.StatefulBrowser(session)
    browser.set_user_agent('Mozilla/5.0')
    browser.open('https://kenpom.com/index.php')

    if 'Cloudflare' in browser.page.title.string:
        raise Exception('Opening kenpom.com failed - request was intercepted by Cloudflare protection')

    # Select and fill the login form
    browser.select_form('form[action="handlers/login_handler.php"]')
    browser['email'] = email
    browser['password'] = password

    # Submit login form
    response = browser.submit_selected()

    if response.status_code != 200 or 'PHPSESSID=' not in response.headers.get('set-cookie', ''):
        raise Exception('Logging in to kenpom.com failed - check that the site is available and your credentials are correct.')
    
    if 'subscription expired' in str(browser.get('https://kenpom.com/index.php').content):
        raise Exception('Logging in to kenpom.com failed - account subscription is expired')

    # Extract cookies from browser to use in the requests session
    for cookie in browser.session.cookies:
        session.cookies.set(cookie.name, cookie.value)
    
    return session
    
def get_table_data(email, password):
    session = login(email, password)
    headers = {
        "User-Agent": "Mozilla/5.0",
        "Referer": "https://kenpom.com/",
        "Accept-Language": "en-US,en;q=0.9",
    }
    

    for season in ['2016','2017','2018','2019','2020','2021','2022','2023']:
        for game in range(0,4000):
            table_url = f"https://kenpom.com/box.php?g={game}&y={season}"
            print(f"Season: {season}, Game: {game}")
            response = session.get(table_url, headers=headers)

            if response.status_code == 200:
                soup = BeautifulSoup(response.content, "html.parser")
                
                # Extract date
                game_info = soup.find("span", class_="game-info")
                print(game_info)
                
                date_str = game_info.text.strip().split(" · ")[0]
                
                # Extract team names and scores from the scoreline
                scoreline = soup.find("h2", class_="scoreline").text.strip()
                print(scoreline)
                # Use regex to extract team names and scores
                team_scores = re.findall(r'([A-Za-z\s.\'&]+?)\s(\d+)', scoreline)
                print(team_scores)
                # Assign parsed values for home and away team names and scores
                away_team, away_score = team_scores[0]
                home_team, home_score = team_scores[1]
                
                # Convert scores to integers
                away_score = int(away_score)
                home_score = int(home_score)

                # Extract half-time scores from the linescore table
                linescore_table = soup.find("table", id="linescore-table2")
                rows = linescore_table.find_all("tr")[1:]  # Skip header row
                
                away_team_1h = int(rows[0].find_all("td")[1].text) + int(rows[0].find_all("td")[2].text)
                home_team_1h = int(rows[1].find_all("td")[1].text) + int(rows[1].find_all("td")[2].text)

                # Create DataFrame
                data = {
                    "Date": [date_str.strip("\n")],
                    "Home Team": [away_team],
                    "Away Team": [home_team],
                    "Home 1H Points": [home_team_1h],
                    "Away 1H Points": [away_team_1h],
                    "Home Full Points": [away_score],
                    "Away Full Points": [home_score],
                }
                
                df = pd.DataFrame(data)
            return df
    #                 except:
    #                     return None
    #     else:
    #         print("Failed to retrieve the page")
    #         return None


    # import time


    # # table_url = f"https://kenpom.com/box.php?g=5&y=2013"
    # # table_data = get_table_data(email, password, table_url)

    # for season in ['2016','2017','2018','2019','2020','2021','2022','2023']:
    #     for game in range(0,4000):
    #         table_url = f"https://kenpom.com/box.php?g={game}&y={season}"
    #         print(f"Season: {season}, Game: {game}")
    #         table_data = get_table_data(email, password, table_url)
    #         df = pd.concat([df, table_data])
    #         time.sleep(3)


In [8]:
import time
df = pd.DataFrame()
email = "nick@ncaainsiders.com"
password = "dIMMITT2021$"

session = login(email, password)
headers = {
        "User-Agent": "Mozilla/5.0",
        "Referer": "https://kenpom.com/",
        "Accept-Language": "en-US,en;q=0.9",
    }
    
for season in ['2024']:
    for game in range(0,1000):
        time.sleep(0.44444)
        table_url = f"https://kenpom.com/box.php?g={game}&y={season}"
        print(f"Season: {season}, Game: {game}")
        response = session.get(table_url, headers=headers)

        if response.status_code == 200:
            try:
                soup = BeautifulSoup(response.content, "html.parser")
                
                # Extract date
                game_info = soup.find("span", class_="game-info")
                print(game_info)
                
                date_str = game_info.text.strip().split(" · ")[0]
                
                # Extract team names and scores from the scoreline
                scoreline = soup.find("h2", class_="scoreline").text.strip()
                print(scoreline)
                # Use regex to extract team names and scores
                team_scores = re.findall(r'([A-Za-z\s.\'&]+?)\s(\d+)', scoreline)
                print(team_scores)
                # Assign parsed values for home and away team names and scores
                away_team, away_score = team_scores[0]
                home_team, home_score = team_scores[1]
                
                # Convert scores to integers
                away_score = int(away_score)
                home_score = int(home_score)

                # Extract half-time scores from the linescore table
                linescore_table = soup.find("table", id="linescore-table2")
                rows = linescore_table.find_all("tr")[1:]  # Skip header row
                
                away_team_1h = int(rows[0].find_all("td")[1].text) + int(rows[0].find_all("td")[2].text)
                home_team_1h = int(rows[1].find_all("td")[1].text) + int(rows[1].find_all("td")[2].text)

                # Create DataFrame
                data = {
                    "Date": [date_str.strip("\n")],
                    "Home Team": [away_team],
                    "Away Team": [home_team],
                    "Home 1H Points": [home_team_1h],
                    "Away 1H Points": [away_team_1h],
                    "Home Full Points": [away_score],
                    "Away Full Points": [home_score],
                }
                
                df1 = pd.DataFrame(data)
                df = pd.concat([df,df1])
                time.sleep(1)
            except:
                None

LinkNotFoundError: 

In [9]:
df.to_csv("predict/2024_games.csv")

In [1]:
import pandas as pd


df = pd.read_csv("games/2016_1344.csv")
df1 = pd.read_csv("games/2016_2019.csv")
df2 = pd.read_csv("games/2017_2250.csv")
df3 = pd.read_csv("games/2018.csv")
df4 = pd.read_csv("games/2019_2020.csv")
df5 = pd.read_csv("games/2020.csv")
df6 = pd.read_csv("games/2021_2022.csv")
df7 = pd.read_csv("games/2022_2023.csv")
df8 = pd.read_csv("games/2023_2024.csv")
df9 = pd.read_csv("games/2024-1.csv")
df10 = pd.read_csv("games/2024-2.csv")


df = pd.concat([df,df1,df2,df3,df4,df5,df6,df7,df8,df9,df10], axis=0)

In [3]:
df.to_csv("games/games.csv")